# Minimal working example for cokringing of a scalar field for geological modeling

The aim of this notebook is to explain in simple steps the scalar-field (or: potential-field) interpolation implemented in GemPy and GeoModeller, based on Lajaunie et al. (1997) and Calcagno et al. (2008).

## Basic principle behind a scalar-field interpolation

The method is based on a zonation principle, i.e. a segmentation of continuous space into discrete zones with similar properties (or other aspects of interest). Typical examples are the distinction of rock units with different lithological features (e.g. different types of sandstones, clay layers, magmatic sequences, etc), but also metamorphic sequences that can be distinguished. Quite generally, it is basically the same concept that is underlying the construction of geological map.

The general idea behind this interpolation method is that geological structures exhibit a specific continuity. As a very basic first principle, we can consider the deposition of sedimentary sequences, for example in a marine environment: we may observe a sequence of more sand-dominated and more clay-dominated units. In a quiet and continuous sedimentation environment, we can assume that we have wide lateral continuity and a "layer-by-layer" deposition.

In the context of geomodeling, we may attempt to describe now these sedimented zones by the interface surfaces between them. It is then obvious that these surfaces should not cross (considering no disturbance during sedimentation). Furthermore, we can assume that the layers show a specific influence on each other: to first order, they can be considered as parallel. And, maybe for a finer distinction, we can assume that a (topographic) variability of in one layer (for example a sea mound, to stay in our example) has a certain upward continuation and can potentially still be apparent (though to a lesser extent) on the next interface above.

We can now take a more abstract view and assume that we do not only two or three interfaces, but more or less a continuous description of subsequent layers, representing a continuous sedimentation process and showing a similar continuous influence on each other, and the layer interfaces can not cross. 

With this intuition, we can describe these layer interfaces as isosurfaces in a scalar field. In our example, we can even interpret the scalar field values as related to a specific depositional age (note that this notion will not generally be possible - nor a requirement). Even further, we can interpret the gradient of this scalar field as orientation values, i.e. measurements of strike/ dip/ dip direction in a geological sequence.

The question now is: how can we interpolate this scalar field from a set of limited observations of surface contact points and orientation measurements?

## Notation and scalar field interpolation

Multiple methods are possible to obtain this scalar field. A very common approach (mostly from the field of image segmentation, but also applied in geophysics) is to use a Level Set formulation (Chan & Vese?). Other previous approaches implemented Radial Basis Functions (RBF's, e.g. the implementation in LeapFrog, see also Hillier, 2014) and a stochastic time interpretation (? Mallet 2004 - the GoCAD implicit modeling approach). We use here a geostatistical method based on (universal) co-kriging, described in Lajaunie et al., 1997.

We denote the 3-D surface in an *implicit* form (therefore also the name of these methods as "implicit geomodeling"), associated with a function $\psi_\alpha$ such that (Lajaunie et al., 1997):

$$C_\alpha = \{x : \psi_a(x) = 0 \}$$


We describe the scalar field that we aim to obtain as a function $T( \vec{x} )$. 

Next, we denote $Z$ as a realization of the (differentiable) random function $\psi$. We now use a kriging method to to estimate $Z$ in the entire domain of interest.

Note: important basic principle: multivariate (co-)kriging, IRK-f model (Matheron!) CHECK!!

However, co-kriging is not "standard" form, as variables are algebraically linked!

More notation:

Gradient data:

$$\frac{\partial Z}{\partial x} (x_i) = G_i^x$$

$$\frac{\partial Z}{\partial y} (x_i) = G_i^y$$

Tangent vector $\tau_i$ is defined through a scalar product:

$$ <\nabla Z(x_i), \tau_i > = 0$$

Points on a single interface belong to a single set $J_k$, $k$ is the index of the interface.

Increments for points on a single interface, the increments must be zero:

$$Z(x_j) - Z(x_{j'}) = 0 \;\;\forall (j,j') \in J_k$$


### The spatial model

As we only consider increments, we can only obtain a (unique) solution when we fix/ select an arbitrary origin $x_0$ and we estimate increments with respect to this origin:

$$Z(x) - Z(x_0) = \sum_{i \in I} \left( \lambda_i G_i^x + \mu_i G_i^y  \right) + \sum_{i' + I'} <\nabla Z(x_{i'}), \tau_{i'} > + \sum_k \sum_{jj' \in \mathcal{P}(J_k)} \lambda_{jj'}[Z(x_j) - Z(x_{j'})] $$ 


$\mathcal{P}(J_k)$ is the set of pairs associated with one interface $J_k$.

We now have to set up the kriging equation to solve for the parameters/ coefficients $\lambda_i, \mu_i, \lambda_{jj'}$ in order to obtain an (explicit?) equation that we can use to determine the potential field value at any point in space.

### Covariance functions

The situation is (a "bit") complicated by the fact that we have to consider all covariances and cross-covariances of each function involved.


(Side note/ Idea/ Check: can the same form of co-kriging be used in the context of posterior-space estimation/ reduced order modeling, etc. - i.e. all the cases where kriging is used, also ML? Because: we could estimate the gradient quite easily using AD-methods and the approach could lead to a more robust estimate? )

For simplicity in the description, we consider an isotropic covariance field. We follow the description in Lajaunie et al. (1997) and denote the covariance of $Z$ as $K_Z$. Furthermore, a vector connecting two points in space is:

$$\vec{h} = \vec{x} - \vec{y}$$

And the components of this vector in $x$- and $y$-direction respectively are $h_x$ and $h_y$. The base covariance function is:

$$K_Z(\vec{h}) = C_Z(r)$$

In order for $Z$ to be differentiable, $K_Z$ must be twice differentiable. Under these conditions (??), the covariances are:

$$K_{ZG^x}(\vec{x} - \vec{y})= Cov(Z(x),Z_x'(y)) = - \frac{h_x}{r} C_Z'(r)$$

Similar:

$$K_{G^x G^y}((\vec{x} - \vec{y})$$

$$K_{G^x}((\vec{x} - \vec{y})$$



### Universality conditions


(Miguel?)

### Dual form

Under the above conditions, we can actually write the kriging equations in the dual form and finally obtain the estimator:

